In [10]:
import librosa
import librosa.display
import IPython.display as ipd
import os
import numpy as np
import pandas as pd
from ipywidgets import Dropdown, Button, Output, VBox
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import joblib
from tensorflow.keras import layers, models
import sounddevice as sd
from scipy.io.wavfile import write
import time
import tempfile

# Function to display the available files in the directory
def list_audio_files(directory):
    return [file for file in os.listdir(directory) if file.endswith('.wav')]

# Feature extraction function with labeled MFCCs
def feature_extraction(file_path):
    """Extracts MFCC features from an audio file."""
    # Load the audio file
    x, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
    
    # Extract MFCC features (22 coefficients)
    mfcc = np.mean(librosa.feature.mfcc(y=x, sr=sample_rate, n_mfcc=22).T, axis=0)
    
    # Label MFCC features (MFCC Coefficient 1, MFCC Coefficient 2, etc.)
    mfcc_labels = [f'MFCC_Coefficient_{i+1}' for i in range(len(mfcc))]
    return mfcc, mfcc_labels

# Directories containing the audio files for Healthy Controls and PwPD
hc_directory = r'C:\Users\S M N RAZA\Downloads\23849127\HC_AH\HC_AH'   # Update with your HC directory path
pwpd_directory = r'C:\Users\S M N RAZA\Downloads\23849127\PD_AH\PD_AH'   # Update with your PwPD directory path

# List all audio files in both directories
hc_audio_files = list_audio_files(hc_directory)
pwpd_audio_files = list_audio_files(pwpd_directory)

# Create an empty DataFrame to store all MFCC features
mfcc_df = pd.DataFrame()

# Extract features from Healthy Controls
for audio_file in hc_audio_files:
    audio_path = os.path.join(hc_directory, audio_file)
    mfcc_features, mfcc_labels = feature_extraction(audio_path)
    temp_df = pd.DataFrame(mfcc_features.reshape(1, -1), columns=mfcc_labels)
    temp_df['Audio_File'] = audio_file
    temp_df['Label'] = 'HC' 
    mfcc_df = pd.concat([mfcc_df, temp_df], ignore_index=True)

# Extract features from PwPD
for audio_file in pwpd_audio_files:
    audio_path = os.path.join(pwpd_directory, audio_file)
    mfcc_features, mfcc_labels = feature_extraction(audio_path)
    temp_df = pd.DataFrame(mfcc_features.reshape(1, -1), columns=mfcc_labels)
    temp_df['Audio_File'] = audio_file
    temp_df['Label'] = 'PwPD'  
    mfcc_df = pd.concat([mfcc_df, temp_df], ignore_index=True)

# Prepare data for model training
X = mfcc_df.drop(columns=['Audio_File', 'Label'])
y = mfcc_df['Label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest model
rfc_model = RandomForestClassifier(random_state=42)
rfc_model.fit(X_train, y_train)

# Initialize and train the SVM model
svm_model = SVC(probability=True)
svm_model.fit(X_train, y_train)

# Initialize and train the KNN model
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)

# Make predictions with all models
rfc_pred = rfc_model.predict(X_test)
svm_pred = svm_model.predict(X_test)
knn_pred = knn_model.predict(X_test)

# Print classification reports and confusion matrices
print("RFC Classification Report:")
print(classification_report(y_test, rfc_pred))

print("RFC Confusion Matrix:")
print(confusion_matrix(y_test, rfc_pred))

print("SVM Classification Report:")
print(classification_report(y_test, svm_pred))

print("SVM Confusion Matrix:")
print(confusion_matrix(y_test, svm_pred))

print("KNN Classification Report:")
print(classification_report(y_test, knn_pred))

print("KNN Confusion Matrix:")
print(confusion_matrix(y_test, knn_pred))

# Save the trained models
joblib.dump(rfc_model, 'rfc_trained_model.joblib')
joblib.dump(svm_model, 'svm_trained_model.joblib')
joblib.dump(knn_model, 'knn_trained_model.joblib')

# Prepare data for CNN
X_cnn = X.values.reshape(X.shape[0], X.shape[1], 1)
X_train_cnn, X_test_cnn, y_train_cnn, y_test_cnn = train_test_split(X_cnn, y, test_size=0.2, random_state=42)


def build_cnn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Conv1D(32, kernel_size=3, activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Conv1D(64, kernel_size=3, activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


cnn_model = build_cnn_model((X_train_cnn.shape[1], 1))
cnn_model.fit(X_train_cnn, y_train_cnn.map({'HC': 0, 'PwPD': 1}).values, epochs=10, batch_size=32, validation_data=(X_test_cnn, y_test_cnn.map({'HC': 0, 'PwPD': 1}).values))


y_pred_cnn = (cnn_model.predict(X_test_cnn) > 0.5).astype("int32").reshape(-1)
print("CNN Classification Report:")
print(classification_report(y_test_cnn.map({'HC': 0, 'PwPD': 1}), y_pred_cnn))
print("CNN Confusion Matrix:")
print(confusion_matrix(y_test_cnn.map({'HC': 0, 'PwPD': 1}), y_pred_cnn))


cnn_model.save('cnn_parkinsons_model.h5') 


def display_audio_and_classify(audio_file, directory):
    audio_path = os.path.join(directory, audio_file)
    print(f"Playing: {audio_file}")
    ipd.display(ipd.Audio(audio_path))
    
    mfcc_features, mfcc_labels = feature_extraction(audio_path)
    mfcc_df_single = pd.DataFrame(mfcc_features.reshape(1, -1), columns=mfcc_labels)
    
    rfc_prediction = rfc_model.predict(mfcc_df_single)[0]
    svm_prediction = svm_model.predict(mfcc_df_single)[0]
    knn_prediction = knn_model.predict(mfcc_df_single)[0]
    cnn_prediction = (cnn_model.predict(mfcc_df_single.values.reshape(1, -1, 1)) > 0.5).astype("int32")[0][0]

    print(f"Random Forest Prediction for {audio_file}: {'Parkinson Disease (PwPD)' if rfc_prediction == 'PwPD' else 'Healthy Control (HC)'}")
    print(f"SVM Prediction for {audio_file}: {'Parkinson Disease (PwPD)' if svm_prediction == 'PwPD' else 'Healthy Control (HC)'}")
    print(f"KNN Prediction for {audio_file}: {'Parkinson Disease (PwPD)' if knn_prediction == 'PwPD' else 'Healthy Control (HC)'}")
    print(f"CNN Prediction for {audio_file}: {'Parkinson Disease (PwPD)' if cnn_prediction == 1 else 'Healthy Control'}")


audio_dropdown = Dropdown(
    options=hc_audio_files + pwpd_audio_files,
    description='Select Audio:'
)
classify_button = Button(description='Classify')
output = Output()

# Function to record live audio 
def record_audio(duration=5, sample_rate=22050):
    print("Recording...")
    audio = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1)
    sd.wait()  # Wait until the recording is finished
    print("Recording complete.")
    
    # Save the audio to a temporary file
    temp_file = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
    write(temp_file.name, sample_rate, audio)  # Save as .wav file
    return temp_file.name

# Button to capture live audio and classify it
record_button = Button(description="Record and Classify Live Audio")

# Button click 
def on_classify_button_clicked(b):
    with output:
        output.clear_output()
        selected_audio = audio_dropdown.value
        directory = hc_directory if selected_audio in hc_audio_files else pwpd_directory
        display_audio_and_classify(selected_audio, directory)

# Button click 
def on_record_button_clicked(b):
    with output:
        output.clear_output()
        temp_file_path = record_audio(duration=5)  # Record 5 seconds of audio
        print("Classifying recorded audio...")
        display_audio_and_classify(temp_file_path, os.path.dirname(temp_file_path))

# Bind the button 
classify_button.on_click(on_classify_button_clicked)
record_button.on_click(on_record_button_clicked)

# Display the widgets
display(VBox([audio_dropdown, classify_button, record_button, output]))





RFC Classification Report:
              precision    recall  f1-score   support

          HC       0.90      0.82      0.86        11
        PwPD       0.71      0.83      0.77         6

    accuracy                           0.82        17
   macro avg       0.81      0.83      0.81        17
weighted avg       0.83      0.82      0.83        17

RFC Confusion Matrix:
[[9 2]
 [1 5]]
SVM Classification Report:
              precision    recall  f1-score   support

          HC       0.00      0.00      0.00        11
        PwPD       0.35      1.00      0.52         6

    accuracy                           0.35        17
   macro avg       0.18      0.50      0.26        17
weighted avg       0.12      0.35      0.18        17

SVM Confusion Matrix:
[[ 0 11]
 [ 0  6]]
KNN Classification Report:
              precision    recall  f1-score   support

          HC       0.67      0.36      0.47        11
        PwPD       0.36      0.67      0.47         6

    accuracy           

c:\Users\S M N RAZA\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\S M N RAZA\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\S M N RAZA\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_

Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 332ms/step - accuracy: 0.4688 - loss: 1.8344 - val_accuracy: 0.6471 - val_loss: 0.9821
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.4479 - loss: 1.4115 - val_accuracy: 0.4706 - val_loss: 0.9013
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.6042 - loss: 0.8341 - val_accuracy: 0.3529 - val_loss: 1.4811
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.5729 - loss: 0.8721 - val_accuracy: 0.5294 - val_loss: 0.7862
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.6146 - loss: 0.6557 - val_accuracy: 0.5882 - val_loss: 0.7374
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.5104 - loss: 0.7619 - val_accuracy: 0.5294 - val_loss: 0.7564
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.7188 - loss: 0.5450 - val_accuracy: 0.4118 - val_loss: 1.1354
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.6146 - loss: 0.6051 - val_accuracy: 0.4706 - val_loss: 1.1066

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step


CNN Classification Report:
              precision    recall  f1-score   support

           0       0.62      0.73      0.67        11
           1       0.25      0.17      0.20         6

    accuracy                           0.53        17
   macro avg       0.43      0.45      0.43        17
weighted avg       0.49      0.53      0.50        17

CNN Confusion Matrix:
[[8 3]
 [5 1]]


In [11]:
# Button click 
def on_classify_button_clicked(b):
    with output:
        output.clear_output()
        selected_audio = audio_dropdown.value
        directory = hc_directory if selected_audio in hc_audio_files else pwpd_directory
        display_audio_and_classify(selected_audio, directory)

# Button click 
def on_record_button_clicked(b):
    with output:
        output.clear_output()
        temp_file_path = record_audio(duration=5)  # Record 5 seconds of audio
        print("Classifying recorded audio...")
        display_audio_and_classify(temp_file_path, os.path.dirname(temp_file_path))

# Bind the button 
classify_button.on_click(on_classify_button_clicked)
record_button.on_click(on_record_button_clicked)

# Display the widgets
display(VBox([audio_dropdown, classify_button, record_button, output]))

In [12]:
from tensorflow.keras import models
